In [ ]:
WITH claims_data AS (
    SELECT
        DATE_ADD(c.created_date, INTERVAL -330 MINUTE) AS locl,
        c.id,
        citd.procedure_text AS others,
        c.hospital_code,
        c.preauth_id,
        cm.routing_reason,
        c.review_reasons,
        car.adjudication_category AS preauth_result,
        co.default_hospital,
        CASE 
            WHEN JSON_SEARCH(JSON_EXTRACT(ifr.policy_data, '$.policy.policyHolderDetails.Riders[*].BenefitName'), 'one', 'Safeguard') IS NOT NULL THEN 'true'
            ELSE 'false'
        END AS contains_safeguard,
        ill.insurer_icd_code
    FROM 
        claims c
    LEFT JOIN 
        claim_illness_treatment_details citd ON c.id = citd.claim_id
    LEFT JOIN 
        procedure_corporate_mapping pcm ON citd.procedure_corporate_mapping_id = pcm.id
    LEFT JOIN 
        procedure_methods pm ON pcm.procedure_method_id = pm.id
    LEFT JOIN 
        procedures p ON p.id = pm.procedure_id
    LEFT JOIN 
        claim_admission_details cad ON c.id = cad.claim_id
    LEFT JOIN 
        claims_metadata cm ON c.id = cm.claim_id
    LEFT JOIN 
        adjudication_scope_config ac ON ac.hospital_code = c.hospital_code 
    LEFT JOIN 
        corporates co ON co.hospital_code = c.hospital_code
    LEFT JOIN 
        claim_adjudication_result car ON car.claim_id = c.id
    LEFT JOIN 
        insurer_fetch_responses ifr ON ifr.claim_id = c.id
    LEFT JOIN 
        illnesses ill ON ill.id = citd.illness_id
    WHERE
        $__timeFilter(c.created_date)
        AND c.old_claim = 0  
        AND car.claim_type = 'PRE_AUTH' 
        AND c.status != 'DUPLICATE'
),

aggregated_data AS (
    SELECT
        DATE_FORMAT(locl, '%Y-%m') AS period,
        COUNT(*) AS Total,
        COUNT(DISTINCT id) AS Top_of_the_Funnel,
        SUM(default_hospital = 1) AS Default_hospital_Adjudication_not_Enabled,
        SUM(default_hospital = 0 AND 
            (routing_reason NOT LIKE '%without%' OR routing_reason IS NULL) AND 
            (routing_reason NOT LIKE '%procedure%' OR routing_reason IS NULL) AND 
            (routing_reason NOT LIKE '%policy%' OR routing_reason IS NULL) AND 
            TRIM(others) IS NULL AND 
            review_reasons LIKE '%ported%') AS Ported_Policy,
        SUM(default_hospital = 0 AND 
            (routing_reason NOT LIKE '%without%' OR routing_reason IS NULL) AND 
            (routing_reason NOT LIKE '%procedure%' OR routing_reason IS NULL) AND 
            (routing_reason NOT LIKE '%policy%' OR routing_reason IS NULL) AND 
            TRIM(others) IS NULL AND 
            (review_reasons NOT LIKE '%ported%' OR review_reasons IS NULL) AND 
            review_reasons LIKE '%first%') AS Dialysis_First_Claim,
        SUM(default_hospital = 0 AND 
            routing_reason LIKE '%procedure%' AND 
            (routing_reason NOT LIKE '%policy%' OR routing_reason IS NULL) AND 
            TRIM(others) IS NULL) AS Procedure_Not_Live,
        SUM(default_hospital = 0 AND 
            routing_reason LIKE '%without%' AND 
            (routing_reason NOT LIKE '%policy%' OR routing_reason IS NULL) AND 
            TRIM(others) IS NULL AND 
            (routing_reason NOT LIKE '%procedure%' OR routing_reason IS NULL)) AS Member_Id_Not_Verified,
        SUM(default_hospital = 0 AND 
            TRIM(others) IS NOT NULL AND 
            (routing_reason NOT LIKE '%policy%' OR routing_reason IS NULL)) AS Procedure_Submitted_in_Others,
        SUM(default_hospital = 0 AND 
            routing_reason LIKE '%Policy%') AS Policy_Product_Not_live,
        SUM(default_hospital = 0 AND 
            preauth_result = 'A' AND 
            (routing_reason NOT LIKE '%without%' OR routing_reason IS NULL) AND 
            (routing_reason NOT LIKE '%procedure%' OR routing_reason IS NULL) AND 
            (routing_reason NOT LIKE '%policy%' OR routing_reason IS NULL) AND 
            TRIM(others) IS NULL AND 
            (review_reasons NOT LIKE '%ported%' OR review_reasons IS NULL) AND 
            (review_reasons NOT LIKE '%first%' OR review_reasons IS NULL)) AS Auto,
        SUM(default_hospital = 0 AND 
            preauth_result != 'A' AND 
            (routing_reason NOT LIKE '%without%' OR routing_reason IS NULL) AND 
            (routing_reason NOT LIKE '%procedure%' OR routing_reason IS NULL) AND 
            (routing_reason NOT LIKE '%policy%' OR routing_reason IS NULL) AND 
            TRIM(others) IS NULL AND 
            (review_reasons NOT LIKE '%ported%' OR review_reasons IS NULL) AND 
            (review_reasons NOT LIKE '%first%' OR review_reasons IS NULL)) AS Manual
    FROM claims_data
    GROUP BY DATE_FORMAT(locl, '%Y-%m')
)

SELECT
    period,
    'Top of the Funnel - Hospitals' AS Metric,
    Top_of_the_Funnel AS Value,
    CASE WHEN Top_of_the_Funnel = 0 THEN 0 ELSE Top_of_the_Funnel / Top_of_the_Funnel * 100 END AS Percentage
FROM aggregated_data

UNION ALL

SELECT
    period,
    'Default hospital - Adjudication not enabled Hospitals' AS Metric,
    Default_hospital_Adjudication_not_Enabled AS Value,
    CASE WHEN Top_of_the_Funnel = 0 THEN 0 ELSE Default_hospital_Adjudication_not_Enabled / Top_of_the_Funnel * 100 END AS Percentage
FROM aggregated_data

UNION ALL

SELECT
    period,
    'Policy Product Not live' AS Metric,
    Policy_Product_Not_live AS Value,
    CASE WHEN Top_of_the_Funnel = 0 THEN 0 ELSE Policy_Product_Not_live / Top_of_the_Funnel * 100 END AS Percentage
FROM aggregated_data

UNION ALL

SELECT
    period,
    'Procedure Submitted in Others' AS Metric,
    Procedure_Submitted_in_Others AS Value,
    CASE WHEN Top_of_the_Funnel = 0 THEN 0 ELSE Procedure_Submitted_in_Others / Top_of_the_Funnel * 100 END AS Percentage
FROM aggregated_data

UNION ALL

SELECT
    period,
    'Procedure Not Live' AS Metric,
    Procedure_Not_Live AS Value,
    CASE WHEN Top_of_the_Funnel = 0 THEN 0 ELSE Procedure_Not_Live / Top_of_the_Funnel * 100 END AS Percentage
FROM aggregated_data

UNION ALL

SELECT
    period,
    'Member Id Not Verified' AS Metric,
    Member_Id_Not_Verified AS Value,
    CASE WHEN Top_of_the_Funnel = 0 THEN 0 ELSE Member_Id_Not_Verified / Top_of_the_Funnel * 100 END AS Percentage
FROM aggregated_data

UNION ALL

SELECT
    period,
    'Ported Policy' AS Metric,
    Ported_Policy AS Value,
    CASE WHEN Top_of_the_Funnel = 0 THEN 0 ELSE Ported_Policy / Top_of_the_Funnel * 100 END AS Percentage
FROM aggregated_data

UNION ALL

SELECT
    period,
    'Dialysis - First Claim' AS Metric,
    Dialysis_First_Claim AS Value,
    CASE WHEN Top_of_the_Funnel = 0 THEN 0 ELSE Dialysis_First_Claim / Top_of_the_Funnel * 100 END AS Percentage
FROM aggregated_data

UNION ALL

SELECT
    period,
    'Manual' AS Metric,
    Manual AS Value,
    CASE WHEN Top_of_the_Funnel = 0 THEN 0 ELSE Manual / Top_of_the_Funnel * 100 END AS Percentage
FROM aggregated_data

UNION ALL

SELECT
    period,
    'Auto' AS Metric,
    Auto AS Value,
    CASE WHEN Top_of_the_Funnel = 0 THEN 0 ELSE Auto / Top_of_the_Funnel * 100 END AS Percentage
FROM aggregated_data

ORDER BY
    period,
    CASE Metric
        WHEN 'Top of the Funnel - Hospitals' THEN 1
        WHEN 'Default hospital - Adjudication not enabled Hospitals' THEN 2
        WHEN 'Policy Product Not live' THEN 3
        WHEN 'Procedure Submitted in Others' THEN 4
        WHEN 'Procedure Not Live' THEN 5
        WHEN 'Member Id Not Verified' THEN 6
        WHEN 'Ported Policy' THEN 7
        WHEN 'Dialysis - First Claim' THEN 8
        WHEN 'Manual' THEN 9
        WHEN 'Auto' THEN 10
        ELSE 11
    END;
